# **App Creation**

An application that allows you to compare all defenders with Acerbi from the perspective of different statistics and also gives a similarity score with the Inter defender may be useful.

## **Installing dependencies**

In [1]:
!pip install streamlit &> /dev/null
!pip install dill &> /dev/null
!pip install soccerplots &> /dev/null

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Firstly, we create the *.py* file which allow us to create ranges for the radar chart.

In [9]:
%%writefile ranges.py
import pandas as pd
import dill

path = '/content/drive/MyDrive/DC_Inter/data/'
with open(path+'not_sorted_defenders.pkl','rb') as f:
    players_stats = dill.load(f)

params = ['progressive_passes_per90', 'tackles_per90',
 'tackles_won_per90', 'blocks_per90', 'shots_blocked_per90', 'interceptions_per90', 'tackles_interceptions_per90', 'clearances_per90', 'ball_recoveries_per90', 'aerial_duels_per90',
'fouls_committed_per90']

def make_ranges(player1, player2, params=params):
  ranges = []
  for x in params:
    a = min(players_stats[(players_stats['Player']== player1) | (players_stats['Player']== player2)][params][x])
    a = a - (a*.25)

    b = max(players_stats[(players_stats['Player']== player1) | (players_stats['Player']== player2)][params][x])
    b = b + (b*.25)

    ranges.append((a,b))
  return ranges

Overwriting ranges.py


Next, we write the code which allow us to create the radar chart.

In [10]:
%%writefile radar.py
from ranges import make_ranges
import pandas as pd
import matplotlib.pyplot as plt
from soccerplots.radar_chart import Radar
import dill

path = '/content/drive/MyDrive/DC_Inter/data/'
with open(path+'not_sorted_defenders.pkl','rb') as f:
    players_stats = dill.load(f)

params = ['progressive_passes_per90', 'tackles_per90',
 'tackles_won_per90', 'blocks_per90', 'shots_blocked_per90', 'interceptions_per90', 'tackles_interceptions_per90', 'clearances_per90', 'ball_recoveries_per90', 'aerial_duels_per90',
'fouls_committed_per90']

def make_radar(player1, player2, params = params):
  player1_values = players_stats[(players_stats['Player']== player1)][params].values.tolist()[0]
  player2_values = players_stats[(players_stats['Player']== player2)][params].values.tolist()[0]
  values = [player1_values, player2_values]
  title = dict(
    title_name=player1,
    title_color='red',
    subtitle_name=players_stats[players_stats['Player'] == player1]['Team'].values[0],
    subtitle_color='red',
    title_name_2=player2,
    title_color_2='blue',
    subtitle_name_2=players_stats[players_stats['Player'] == player2]['Team'].values[0],
    subtitle_color_2='blue',
    title_fontsize=18,
    subtitle_fontsize=15,
)
  endnote = f"Similarity score: {round(players_stats[players_stats['Player'] == player2]['distance from acerbi'].values[0],3)}\nVisualization made by Cristian Ceccarelli\nAll the stats are per90 and refer to the 2022/2023 season\nThe lower the similarity score, the more similar the players are\nNOTE: The similarity score was computed on ALL the statistics from which you can choose ."
  radar = Radar()
  return radar.plot_radar(ranges = make_ranges(player1, player2, params),
                          params =params,
                          values = values,
                          title = title,
                          endnote=endnote,
                          radar_color = ['red', 'blue'],
                          compare = True)

Overwriting radar.py


Finally we write the file which allow us to create the webpage.

In [25]:
%%writefile app.py
import streamlit as st
from radar import make_radar
import dill

path = '/content/drive/MyDrive/DC_Inter/data/'
with open(path+'not_sorted_defenders.pkl','rb') as f:
    players_stats = dill.load(f)


params = ['progressive_passes_per90', 'tackles_per90',
 'tackles_won_per90', 'blocks_per90', 'shots_blocked_per90', 'interceptions_per90', 'tackles_interceptions_per90', 'clearances_per90', 'ball_recoveries_per90', 'aerial_duels_per90',
'fouls_committed_per90']



st.header('Compare defenders with Francesco Acerbi', divider='red')
st.write("Choose a defender to compare with Acerbi and at least 3 statistics on which to compare the two players.")
league = st.selectbox(label = 'Select a league', options = list(players_stats['League'].unique()))

if league:
  team = st.selectbox(label = 'Select a team', options = list(players_stats[players_stats['League'] == league]['Team'].unique()))
  if team:
    player = st.selectbox(label = 'Select a player',options=list(players_stats[players_stats['Team'] == team]['Player'].unique()))

    if player:
      st.write(players_stats[players_stats['Player'] == player][['Player','Team','Born', 'Nation'] + params])
      input_params = list(st.multiselect(label = 'Select 3 or more stats on which you want to compare the players', options = list(players_stats.columns)[10:]))
      if len(input_params) >= 3:
        st.pyplot(fig = make_radar('Francesco Acerbi', player,input_params)[0])
      else:
        st.pyplot(fig = make_radar('Francesco Acerbi', player, params)[0])

st.write("The 5 players most similar to Francesco Acerbi are:")
st.write(players_stats.sort_values(by='distance from acerbi')[['Player', 'Team','distance from acerbi']][1:6])


Overwriting app.py


## **Accessing the webpage**

In [23]:
!wget -q -O  - ipv4.icanhazip.com

34.91.190.61


In [27]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501

npx: installed 22 in 2.447s
your url is: https://neat-owls-begin.loca.lt
^C
